In [26]:
!pip install -q \
    datasets \
    transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.9 MB/s eta 0:00:00


In [27]:
from datasets import load_dataset
raw_datasets = load_dataset("code_search_net", "python")

  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [29]:
#raw_datasets["train"][123456]

In [30]:
raw_datasets["train"][123456]["whole_func_string"]

"def fetch_genome(genome_id):\n    '''Acquire a genome from Entrez\n\n    '''\n    # TODO: Can strandedness by found in fetched genome attributes?\n    # TODO: skip read/write step?\n    # Using a dummy email for now - does this violate NCBI guidelines?\n    email = 'loremipsum@gmail.com'\n    Entrez.email = email\n\n    print 'Downloading Genome...'\n    handle = Entrez.efetch(db='nucleotide', id=str(genome_id), rettype='gb',\n                           retmode='text')\n    print 'Genome Downloaded...'\n    tmpfile = os.path.join(mkdtemp(), 'tmp.gb')\n    with open(tmpfile, 'w') as f:\n        f.write(handle.read())\n    genome = coral.seqio.read_dna(tmpfile)\n\n    return genome"

In [31]:
print(raw_datasets["train"][123456]["whole_func_string"])

def fetch_genome(genome_id):
    '''Acquire a genome from Entrez

    '''
    # TODO: Can strandedness by found in fetched genome attributes?
    # TODO: skip read/write step?
    # Using a dummy email for now - does this violate NCBI guidelines?
    email = 'loremipsum@gmail.com'
    Entrez.email = email

    print 'Downloading Genome...'
    handle = Entrez.efetch(db='nucleotide', id=str(genome_id), rettype='gb',
                           retmode='text')
    print 'Genome Downloaded...'
    tmpfile = os.path.join(mkdtemp(), 'tmp.gb')
    with open(tmpfile, 'w') as f:
        f.write(handle.read())
    genome = coral.seqio.read_dna(tmpfile)

    return genome


In [32]:
whole = (raw_datasets["train"][123456]["func_name"]
    + raw_datasets["train"][123456]["func_documentation_string"]
    + raw_datasets["train"][123456]["func_code_string"]
)
print(whole)

fetch_genomeAcquire a genome from Entrezdef fetch_genome(genome_id):
    '''Acquire a genome from Entrez

    '''
    # TODO: Can strandedness by found in fetched genome attributes?
    # TODO: skip read/write step?
    # Using a dummy email for now - does this violate NCBI guidelines?
    email = 'loremipsum@gmail.com'
    Entrez.email = email

    print 'Downloading Genome...'
    handle = Entrez.efetch(db='nucleotide', id=str(genome_id), rettype='gb',
                           retmode='text')
    print 'Genome Downloaded...'
    tmpfile = os.path.join(mkdtemp(), 'tmp.gb')
    with open(tmpfile, 'w') as f:
        f.write(handle.read())
    genome = coral.seqio.read_dna(tmpfile)

    return genome


In [33]:
#raw_datasets["train"][123456]["func_name"]
raw_datasets["train"][123456]["func_documentation_string"]

'Acquire a genome from Entrez'

In [34]:
raw_datasets["train"][123456]["func_code_string"] == \
raw_datasets["train"][123456]["whole_func_string"]

True

In [35]:
STEP = 1000
def get_training_corpus(step=STEP):
    train_data = raw_datasets["train"]
    for idx in range(0, len(train_data), step):
        samples = train_data[idx: idx+step]
        yield samples["whole_func_string"]

In [36]:
from tqdm.auto import tqdm

In [37]:
len(next(get_training_corpus()))

1000

In [38]:
training_corpus = get_training_corpus()
length = len(raw_datasets["train"])
length = length // STEP + (length % STEP > 0)
print(f'{length = }')
pbar = tqdm(total=length)
for sample in training_corpus:
    pbar.update(1)
    continue
len(sample)

length = 413


  0%|          | 0/413 [00:00<?, ?it/s]

178

In [39]:
type(next(get_training_corpus()))

list

In [40]:
print(next(get_training_corpus())[0])

async def ensure_open(self) -> None:
        """
        Check that the WebSocket connection is open.

        Raise :exc:`~websockets.exceptions.ConnectionClosed` if it isn't.

        """
        # Handle cases from most common to least common for performance.
        if self.state is State.OPEN:
            # If self.transfer_data_task exited without a closing handshake,
            # self.close_connection_task may be closing it, going straight
            # from OPEN to CLOSED.
            if self.transfer_data_task.done():
                await asyncio.shield(self.close_connection_task)
                raise ConnectionClosed(
                    self.close_code, self.close_reason
                ) from self.transfer_data_exc
            else:
                return

        if self.state is State.CLOSED:
            raise ConnectionClosed(
                self.close_code, self.close_reason
            ) from self.transfer_data_exc

        if self.state is State.CLOSING:
         

So it looks like that `get_training_corpus`
- Returns a generator
- Most of the time yields a list (of length `STEP`) of strings, each string being the code of some Python function in GitHub
- Except the last "batch" yielded: It'll have some kind of a residual length, the remainder of dividing `len(raw_datasets["train"])` by `STEP`

In [42]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
old_tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [47]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
print(tokens)

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


- `'Ċ'`: newline
- `'Ġ'`: space